<img src="data/ima/herraa.jpg" alt="Imagen creada con inteligencia artificial y editada con Microsoft Paint" style="border-radius: 15px;">

*Imagen creada con inteligencia artificial y editada con Microsoft Paint*


## BIBLIOTECAS USADAS:

In [42]:
import numpy as np
import pandas as pd 
from scipy.stats import f_oneway

## DATASETS DE PRUEBA.

In [39]:
df1 = pd.read_csv('./data/datasets/lifesat_full.csv', sep=',')
df2 = pd.read_csv('./data/datasets/pima_indians.csv', sep=',') #Target 'class'
df3 = pd.read_csv('./data/datasets/temps.csv', sep=',')
df4 = pd.read_csv('./data/datasets/titanic.csv', sep=',') #Target 'alive'
df5 = pd.read_csv('./data/datasets/CarPrice_Assignment.csv', sep=',') # target 'Price... o eso creo.  
df6 = pd.read_csv('./data/datasets/bank-full.csv', sep=';') #El target parce que es 'y'  
df7 = pd.read_csv('./data/datasets/dataset_viajes_jun23.csv', sep=',') #El target, 'aircompany' por ejemplo.

## Funcion: get_features_num_classification

In [ ]:
def get_features_num_classification(df, target_col, pvalue= 0.05): 
    columnas_numericas = []
    if isinstance(df, pd.DataFrame):
        print(f"{df} es un DataFrame")
    else:
        print(f"{df} no es un dataframe, repase la llamada a la función")
        return 

    if target_col not in df.columns:
        print(f"{target_col} no es una columna del dataframe {df}")
        return
    
    tipos_validos = ['category', 'int64', 'bool', 'object']
    if not pd.api.types.is_dtype_equal(df[target_col], tipos_validos):
        print(f"La columna '{target_col}' no es categórica ni disteta, revisa la llamada a la funcion.")
        return 
    
    if pd.api.types.is_integer_dtype(df[target_col]) and df[target_col].nunique() >= 15:
        print(f"¡OJO!.. La columna '{target_col}' tiene alta cardinalidad (> 15 categorías).")

    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            columnas_numericas.append(col)
 